In [1]:
import keras
from keras import regularizers
import keras.backend as K
import tensorflow as tf
import numpy as np

import utils
import loggingreporter 

cfg = {}
cfg['SGD_BATCHSIZE']    = 64
cfg['SGD_LEARNINGRATE'] = 0.001
cfg['NUM_EPOCHS']       = 25000   # Anver changed from 10000
cfg['FULL_MI']          = True

# cfg['ACTIVATION'] = 'tanh'
cfg['ACTIVATION'] = 'relu'   ## Anver changed to tanh from relu

# How many hidden neurons to put into each of the layers
cfg['LAYER_DIMS'] = [16, 16, 8]
ARCH_NAME =  '-'.join(map(str,cfg['LAYER_DIMS']))

# No validation data is created. Only TRAIN and TEST
trn, tst = utils.get_pokeman()

# Where to save activation and weights data
cfg['SAVE_DIR'] = 'rawdata/' + cfg['ACTIVATION'] + '_' + ARCH_NAME 


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
#trn

In [3]:
# We should keep the initialization and optimizer the same across different experiments to remove other potential influences
# IBnet uses truncatedNormal and Adam

input_layer  = keras.layers.Input((trn.X.shape[1],))

clayer = input_layer

# Try L2norm, dropout here
for n in cfg['LAYER_DIMS']:
    clayer = keras.layers.Dense(n, 
                                activation=cfg['ACTIVATION'],
                                kernel_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=1/np.sqrt(float(n)), seed=10),
                                bias_initializer='zeros',
                                kernel_regularizer=regularizers.l2(0.001) # add L2 norm regularization #Anver Added regularizers
                               )(clayer)
#     clayer = keras.layers.Dropout(0.1)(clayer)
output_layer = keras.layers.Dense(trn.nb_classes, activation='softmax')(clayer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# sgd = keras.optimizers.SGD(lr=cfg['SGD_LEARNINGRATE'], decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1.0, clipvalue=0.5)
sgd = keras.optimizers.SGD(lr=cfg['SGD_LEARNINGRATE'], momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 27        
Total params: 515
Trainable params: 515
Non-trainable params: 0
_________________________________________________________________


In [4]:
def do_report(epoch):
    # Only log activity for some epochs.  Mainly this is to make things run faster.
    if epoch < 20:       # Log for all first 20 epochs
        return True
    elif epoch < 100:    # Then for every 5th epoch
        return (epoch % 5 == 0)
    elif epoch < 200:    # Then every 10th
        return (epoch % 10 == 0)
    else:                # Then every 100th
        return (epoch % 100 == 0)
    
reporter = loggingreporter.LoggingReporter(cfg=cfg, 
                                          trn=trn, 
                                          tst=tst, 
                                          do_save_func=do_report)
r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 2, 
              batch_size = cfg['SGD_BATCHSIZE'],
              epochs     = cfg['NUM_EPOCHS'],
              validation_split = 0.2,
              shuffle = True,
              # validation_data=(tst.X, tst.Y),
              callbacks  = [reporter,])


Train on 1152 samples, validate on 288 samples
Making directory rawdata/relu_16-16-8
Epoch 1/25000
 - 0s - loss: 1.1216 - acc: 0.3385 - val_loss: 1.1218 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000000
Epoch 2/25000
 - 0s - loss: 1.1215 - acc: 0.3385 - val_loss: 1.1218 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000001
Epoch 3/25000
 - 0s - loss: 1.1215 - acc: 0.3385 - val_loss: 1.1219 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000002
Epoch 4/25000
 - 0s - loss: 1.1214 - acc: 0.3385 - val_loss: 1.1221 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000003
Epoch 5/25000
 - 0s - loss: 1.1214 - acc: 0.3385 - val_loss: 1.1221 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000004
Epoch 6/25000
 - 0s - loss: 1.1213 - acc: 0.3385 - val_loss: 1.1223 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000005
Epoch 7/25000
 - 0s - loss: 1.1213 - acc: 0.3385 - val_loss: 1.1223 - val_acc: 0.3194
Saving rawdata/relu_16-16-8/epoch00000006
Epoch 8/25000
 - 0s

Epoch 79/25000
 - 0s - loss: 1.1195 - acc: 0.3394 - val_loss: 1.1220 - val_acc: 0.3056
Epoch 80/25000
 - 0s - loss: 1.1195 - acc: 0.3394 - val_loss: 1.1220 - val_acc: 0.3056
Epoch 81/25000
 - 0s - loss: 1.1195 - acc: 0.3394 - val_loss: 1.1220 - val_acc: 0.3056
Saving rawdata/relu_16-16-8/epoch00000080
Epoch 82/25000
 - 0s - loss: 1.1195 - acc: 0.3212 - val_loss: 1.1220 - val_acc: 0.3056
Epoch 83/25000
 - 0s - loss: 1.1195 - acc: 0.3411 - val_loss: 1.1219 - val_acc: 0.3438
Epoch 84/25000
 - 0s - loss: 1.1195 - acc: 0.3290 - val_loss: 1.1219 - val_acc: 0.3056
Epoch 85/25000
 - 0s - loss: 1.1194 - acc: 0.3394 - val_loss: 1.1219 - val_acc: 0.3056
Epoch 86/25000
 - 0s - loss: 1.1194 - acc: 0.3403 - val_loss: 1.1219 - val_acc: 0.3611
Saving rawdata/relu_16-16-8/epoch00000085
Epoch 87/25000
 - 0s - loss: 1.1194 - acc: 0.3299 - val_loss: 1.1219 - val_acc: 0.3056
Epoch 88/25000
 - 0s - loss: 1.1194 - acc: 0.3394 - val_loss: 1.1219 - val_acc: 0.3056
Epoch 89/25000
 - 0s - loss: 1.1194 - acc: 0.3

Epoch 168/25000
 - 0s - loss: 1.1177 - acc: 0.3394 - val_loss: 1.1203 - val_acc: 0.3056
Epoch 169/25000
 - 0s - loss: 1.1177 - acc: 0.3394 - val_loss: 1.1201 - val_acc: 0.3056
Epoch 170/25000
 - 0s - loss: 1.1177 - acc: 0.3394 - val_loss: 1.1201 - val_acc: 0.3056
Epoch 171/25000
 - 0s - loss: 1.1177 - acc: 0.3429 - val_loss: 1.1201 - val_acc: 0.4097
Saving rawdata/relu_16-16-8/epoch00000170
Epoch 172/25000
 - 0s - loss: 1.1177 - acc: 0.3872 - val_loss: 1.1201 - val_acc: 0.3056
Epoch 173/25000
 - 0s - loss: 1.1176 - acc: 0.3394 - val_loss: 1.1201 - val_acc: 0.3056
Epoch 174/25000
 - 0s - loss: 1.1176 - acc: 0.3394 - val_loss: 1.1201 - val_acc: 0.3056
Epoch 175/25000
 - 0s - loss: 1.1176 - acc: 0.3438 - val_loss: 1.1200 - val_acc: 0.3194
Epoch 176/25000
 - 0s - loss: 1.1176 - acc: 0.3568 - val_loss: 1.1200 - val_acc: 0.3333
Epoch 177/25000
 - 0s - loss: 1.1175 - acc: 0.4062 - val_loss: 1.1200 - val_acc: 0.3611
Epoch 178/25000
 - 0s - loss: 1.1175 - acc: 0.3672 - val_loss: 1.1200 - val_ac

Epoch 260/25000
 - 0s - loss: 1.1159 - acc: 0.3394 - val_loss: 1.1184 - val_acc: 0.3056
Epoch 261/25000
 - 0s - loss: 1.1159 - acc: 0.3394 - val_loss: 1.1184 - val_acc: 0.3056
Epoch 262/25000
 - 0s - loss: 1.1159 - acc: 0.3646 - val_loss: 1.1184 - val_acc: 0.2986
Epoch 263/25000
 - 0s - loss: 1.1158 - acc: 0.3368 - val_loss: 1.1184 - val_acc: 0.3056
Epoch 264/25000
 - 0s - loss: 1.1158 - acc: 0.3455 - val_loss: 1.1183 - val_acc: 0.2882
Epoch 265/25000
 - 0s - loss: 1.1158 - acc: 0.3542 - val_loss: 1.1183 - val_acc: 0.3264
Epoch 266/25000
 - 0s - loss: 1.1158 - acc: 0.4019 - val_loss: 1.1184 - val_acc: 0.3576
Epoch 267/25000
 - 0s - loss: 1.1158 - acc: 0.3767 - val_loss: 1.1181 - val_acc: 0.2674
Epoch 268/25000
 - 0s - loss: 1.1157 - acc: 0.3542 - val_loss: 1.1182 - val_acc: 0.2847
Epoch 269/25000
 - 0s - loss: 1.1157 - acc: 0.3368 - val_loss: 1.1182 - val_acc: 0.3056
Epoch 270/25000
 - 0s - loss: 1.1157 - acc: 0.3620 - val_loss: 1.1181 - val_acc: 0.3264
Epoch 271/25000
 - 0s - loss: 1.

Epoch 353/25000
 - 0s - loss: 1.1140 - acc: 0.3602 - val_loss: 1.1164 - val_acc: 0.3576
Epoch 354/25000
 - 0s - loss: 1.1139 - acc: 0.3385 - val_loss: 1.1163 - val_acc: 0.2535
Epoch 355/25000
 - 0s - loss: 1.1139 - acc: 0.3707 - val_loss: 1.1162 - val_acc: 0.3542
Epoch 356/25000
 - 0s - loss: 1.1139 - acc: 0.3611 - val_loss: 1.1163 - val_acc: 0.2361
Epoch 357/25000
 - 0s - loss: 1.1139 - acc: 0.3446 - val_loss: 1.1162 - val_acc: 0.2292
Epoch 358/25000
 - 0s - loss: 1.1138 - acc: 0.2769 - val_loss: 1.1162 - val_acc: 0.3021
Epoch 359/25000
 - 0s - loss: 1.1138 - acc: 0.4045 - val_loss: 1.1162 - val_acc: 0.3576
Epoch 360/25000
 - 0s - loss: 1.1138 - acc: 0.3351 - val_loss: 1.1162 - val_acc: 0.2326
Epoch 361/25000
 - 0s - loss: 1.1138 - acc: 0.2891 - val_loss: 1.1162 - val_acc: 0.2743
Epoch 362/25000
 - 0s - loss: 1.1137 - acc: 0.3082 - val_loss: 1.1162 - val_acc: 0.3056
Epoch 363/25000
 - 0s - loss: 1.1137 - acc: 0.3620 - val_loss: 1.1162 - val_acc: 0.3576
Epoch 364/25000
 - 0s - loss: 1.

...

Epoch 24585/25000
 - 0s - loss: 0.7882 - acc: 0.6762 - val_loss: 0.8232 - val_acc: 0.6285
Epoch 24586/25000
 - 0s - loss: 0.7947 - acc: 0.6649 - val_loss: 0.7655 - val_acc: 0.6910
Epoch 24587/25000
 - 0s - loss: 0.7927 - acc: 0.6536 - val_loss: 0.7723 - val_acc: 0.6910
Epoch 24588/25000
 - 0s - loss: 0.7902 - acc: 0.6649 - val_loss: 0.7563 - val_acc: 0.6562
Epoch 24589/25000
 - 0s - loss: 0.7917 - acc: 0.6641 - val_loss: 0.7617 - val_acc: 0.6944
Epoch 24590/25000
 - 0s - loss: 0.7966 - acc: 0.6589 - val_loss: 0.7605 - val_acc: 0.6597
Epoch 24591/25000
 - 0s - loss: 0.7918 - acc: 0.6597 - val_loss: 0.7603 - val_acc: 0.6597
Epoch 24592/25000
 - 0s - loss: 0.7880 - acc: 0.6649 - val_loss: 0.7688 - val_acc: 0.6910
Epoch 24593/25000
 - 0s - loss: 0.7953 - acc: 0.6562 - val_loss: 0.7897 - val_acc: 0.6806
Epoch 24594/25000
 - 0s - loss: 0.7957 - acc: 0.6554 - val_loss: 0.7613 - val_acc: 0.6875
Epoch 24595/25000
 - 0s - loss: 0.7916 - acc: 0.6562 - val_loss: 0.7579 - val_acc: 0.6562
Epoch 2459

Epoch 24676/25000
 - 0s - loss: 0.7910 - acc: 0.6649 - val_loss: 0.7578 - val_acc: 0.6562
Epoch 24677/25000
 - 0s - loss: 0.7922 - acc: 0.6623 - val_loss: 0.7590 - val_acc: 0.6597
Epoch 24678/25000
 - 0s - loss: 0.7906 - acc: 0.6580 - val_loss: 0.7611 - val_acc: 0.6944
Epoch 24679/25000
 - 0s - loss: 0.7895 - acc: 0.6658 - val_loss: 0.7591 - val_acc: 0.6701
Epoch 24680/25000
 - 0s - loss: 0.7880 - acc: 0.6684 - val_loss: 0.7566 - val_acc: 0.6562
Epoch 24681/25000
 - 0s - loss: 0.7913 - acc: 0.6623 - val_loss: 0.7587 - val_acc: 0.6667
Epoch 24682/25000
 - 0s - loss: 0.7887 - acc: 0.6502 - val_loss: 0.7733 - val_acc: 0.6389
Epoch 24683/25000
 - 0s - loss: 0.7928 - acc: 0.6606 - val_loss: 0.7598 - val_acc: 0.6562
Epoch 24684/25000
 - 0s - loss: 0.7882 - acc: 0.6623 - val_loss: 0.7639 - val_acc: 0.6910
Epoch 24685/25000
 - 0s - loss: 0.7914 - acc: 0.6606 - val_loss: 0.7605 - val_acc: 0.6910
Epoch 24686/25000
 - 0s - loss: 0.7909 - acc: 0.6606 - val_loss: 0.7689 - val_acc: 0.6910
Epoch 2468

Epoch 24767/25000
 - 0s - loss: 0.7887 - acc: 0.6710 - val_loss: 0.7680 - val_acc: 0.6910
Epoch 24768/25000
 - 0s - loss: 0.7895 - acc: 0.6649 - val_loss: 0.7688 - val_acc: 0.6910
Epoch 24769/25000
 - 0s - loss: 0.7944 - acc: 0.6562 - val_loss: 0.8112 - val_acc: 0.6771
Epoch 24770/25000
 - 0s - loss: 0.7926 - acc: 0.6684 - val_loss: 0.7625 - val_acc: 0.6910
Epoch 24771/25000
 - 0s - loss: 0.7892 - acc: 0.6606 - val_loss: 0.7805 - val_acc: 0.6806
Epoch 24772/25000
 - 0s - loss: 0.7929 - acc: 0.6675 - val_loss: 0.7718 - val_acc: 0.6910
Epoch 24773/25000
 - 0s - loss: 0.7878 - acc: 0.6684 - val_loss: 0.7616 - val_acc: 0.6910
Epoch 24774/25000
 - 0s - loss: 0.7946 - acc: 0.6580 - val_loss: 0.7734 - val_acc: 0.6910
Epoch 24775/25000
 - 0s - loss: 0.7950 - acc: 0.6554 - val_loss: 0.7801 - val_acc: 0.6806
Epoch 24776/25000
 - 0s - loss: 0.7917 - acc: 0.6710 - val_loss: 0.7572 - val_acc: 0.6701
Epoch 24777/25000
 - 0s - loss: 0.7890 - acc: 0.6641 - val_loss: 0.7623 - val_acc: 0.6910
Epoch 2477

Epoch 24858/25000
 - 0s - loss: 0.7880 - acc: 0.6684 - val_loss: 0.7582 - val_acc: 0.6562
Epoch 24859/25000
 - 0s - loss: 0.7870 - acc: 0.6623 - val_loss: 0.7583 - val_acc: 0.6562
Epoch 24860/25000
 - 0s - loss: 0.7940 - acc: 0.6580 - val_loss: 0.7717 - val_acc: 0.6910
Epoch 24861/25000
 - 0s - loss: 0.7893 - acc: 0.6641 - val_loss: 0.7743 - val_acc: 0.6910
Epoch 24862/25000
 - 0s - loss: 0.7894 - acc: 0.6658 - val_loss: 0.7582 - val_acc: 0.6562
Epoch 24863/25000
 - 0s - loss: 0.7890 - acc: 0.6684 - val_loss: 0.7602 - val_acc: 0.6597
Epoch 24864/25000
 - 0s - loss: 0.7937 - acc: 0.6632 - val_loss: 0.7645 - val_acc: 0.6910
Epoch 24865/25000
 - 0s - loss: 0.7910 - acc: 0.6580 - val_loss: 0.7755 - val_acc: 0.6910
Epoch 24866/25000
 - 0s - loss: 0.7962 - acc: 0.6623 - val_loss: 0.7668 - val_acc: 0.6910
Epoch 24867/25000
 - 0s - loss: 0.7872 - acc: 0.6675 - val_loss: 0.7615 - val_acc: 0.6562
Epoch 24868/25000
 - 0s - loss: 0.7957 - acc: 0.6606 - val_loss: 0.7585 - val_acc: 0.6562
Epoch 2486

Epoch 24949/25000
 - 0s - loss: 0.7904 - acc: 0.6658 - val_loss: 0.7831 - val_acc: 0.6806
Epoch 24950/25000
 - 0s - loss: 0.7912 - acc: 0.6693 - val_loss: 0.7710 - val_acc: 0.6910
Epoch 24951/25000
 - 0s - loss: 0.7882 - acc: 0.6684 - val_loss: 0.7606 - val_acc: 0.6597
Epoch 24952/25000
 - 0s - loss: 0.7927 - acc: 0.6701 - val_loss: 0.7582 - val_acc: 0.6597
Epoch 24953/25000
 - 0s - loss: 0.7876 - acc: 0.6658 - val_loss: 0.7586 - val_acc: 0.6597
Epoch 24954/25000
 - 0s - loss: 0.7907 - acc: 0.6589 - val_loss: 0.7574 - val_acc: 0.6562
Epoch 24955/25000
 - 0s - loss: 0.7883 - acc: 0.6667 - val_loss: 0.7830 - val_acc: 0.6806
Epoch 24956/25000
 - 0s - loss: 0.7901 - acc: 0.6684 - val_loss: 0.7659 - val_acc: 0.6458
Epoch 24957/25000
 - 0s - loss: 0.7911 - acc: 0.6632 - val_loss: 0.7692 - val_acc: 0.6424
Epoch 24958/25000
 - 0s - loss: 0.7920 - acc: 0.6597 - val_loss: 0.7583 - val_acc: 0.6701
Epoch 24959/25000
 - 0s - loss: 0.7889 - acc: 0.6580 - val_loss: 0.7935 - val_acc: 0.6806
Epoch 2496

In [5]:
# Takes insane time to run on my own laptop, but it works